# Scrapper

## Required libraries

In [125]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

## Utilities

In [126]:
def get_tweet_data(card):
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_count)
    return tweet

## Login process

In [129]:
TWITTER_ACCOUNT_NAME_OR_MOBILE_PHONE = "3164917357"

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.get('https://twitter.com/i/flow/login')
sleep(10)

# Send username through specified field
username_field = driver.find_element(By.XPATH, '//input[@autocomplete="username"]')
username_field.send_keys(TWITTER_ACCOUNT_NAME_OR_MOBILE_PHONE)
username_field.send_keys(Keys.RETURN)

# Do the same but with the password
my_password = getpass()
password_field = driver.find_element(By.XPATH, '//input[@autocomplete="current-password"]')
password_field.send_keys(my_password)
password_field.send_keys(Keys.RETURN)
sleep(5)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_mac64.zip
Driver has been saved in cache [/Users/fernandoa/.wdm/drivers/chromedriver/mac64/97.0.4692.71]


 ··············


## Tweet extraction

In [216]:
profiles = {
    'Petro': 'https://twitter.com/petrogustavo',
    'Fajardo': 'https://twitter.com/sergio_fajardo',
    'Uribe': 'https://twitter.com/AlvaroUribeVel'
}

tweets_info = set()
SCROLL_THRESHOLD = 3

In [217]:
for candidate in profiles.keys():
    driver.get(profile)
    sleep(3)
    liked = driver.find_element(By.LINK_TEXT, 'Me gusta').click()
    sleep(3)
    tweets = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')

    for tweet in tweets:
        content = tweet.find_element(By.XPATH, './/div[2]/div[2]/div[2]')
        text = ''.join([piece.text for piece in content.find_elements(By.XPATH, './/span')])
        
        if text not in tweets_info:
            tweets_info.add((text, candidate))